In [ ]:
import pandas                  as pd
import numpy                   as np
import pathlib
import urllib

## Save the file locally if it's not there (makes tweaking faster)

In [ ]:
DATA_URL = 'https://www.maine.gov/sos/cec/elec/data/absentee-voter-file.txt'
INPUT_FILENAME = './data/absentee-voter-file.txt'
# Supported formats: 'xlsx', 'inline'
OUTPUT_FORMAT = 'xlsx'
OUTPUT_FILENAME = './output/absentee-analysis-returns-full.xlsx'

In [ ]:
# Download file if it doesn't already exist. Create directories if necessary.
input_path = pathlib.Path(INPUT_FILENAME)
input_path.parents[0].mkdir(parents=True, exist_ok=True)
if not input_path.exists():
    urllib.request.urlretrieve(DATA_URL, input_path)

In [ ]:
# Read back the downloaded file.
raw_data = pd.read_csv(input_path, sep='|', skipfooter=5, engine='python')
raw_data.head()

## Some constants. Honestly more for reference at the moment.

In [ ]:
REJECT_REASONS = {
    'ANC': 'AIDE CERTIFICATE NOT PROPERLY COMPLETED',
    'BND': 'BALLOT NOT RECEIVED BY DEADLINE DATE',
    'BRU': 'BALLOT RETURNED AS UNDELIVERABLE',
    'DBR': 'DUPLICATE BALLOT RECEIVED',
    'ENS': 'ENVELOPE NOT SIGNED BY VOTER',
    'NEN': 'NOT ENROLLED WHEN NECESSARY',
    'NRV': 'NOT A REGISTERED VOTER',
    'OTH': 'OTHER',
    'RAD': 'RECEIVED AFTER DEADLINE DATE',
    'SBV': 'SPOILED BY VOTER',
    'SNM': 'SIGNATURE ON APPLICATION AND ENVELOPE DO NOT MATCH',
    'VIP': 'VOTED IN PERSON',
    'WNC': 'WITNESS CERTIFICATE NOT PROPERLY COMPLETED',
}

COLUMN_NAMES = {
    'MUNICIPALITY': 'MUNCIPALITY',
    'CH': 'CHALLENGE INDICATOR',
    'DES': 'SPECIAL DESIGNATORS',
    'VOTER ID': 'VOTER ID (VOTER RECORD #)',
    'P': 'ENROLLMENT',
    'W/P': 'WARD/PRECINCT',
    'CG': 'CONGRESSIONAL DISTRICT',
    'SS': 'STATE SENATE',
    'SR': 'STATE REPRESENTATIVE',
    'CC': 'COUNTY COMMISSIONER DISTRICT',
    'REQTYPE': 'REQUEST RECEIVED BY',
    'REQDATE': 'REQUEST',
    'REQDEN':  'ACCEPT/REJECTED',
    'ISSTYPE': 'BALLOT SENT BY',
    'ISSDATE': 'BALLOT SENT',
    'RECTYPE': 'BALLOT RECEIVED BY',
    'RECDATE': 'BALLOT RECEIVED',
    'DUP': 'DUPLICATE BALLOT SEQUENCE NUMBER',
    'ACC OR REJ': 'ACCEPTED/REJECTED',
    'REJRSN': 'REJECT REASON',
}

PARTY_CODES = {
    'B': 'AMERICAN DELTA',
    'D': 'DEMOCRATIC',
    'G': 'GREEN INDEPENDENT',
    'L': 'LIBERTARIAN',
    'O': 'OTHER QUALIFYING PARTY',
    'P': 'PROGRESSIVE',
    'R': 'REPUBLICAN',
    'U': 'UNENROLLED',
}

## Group Data on `MUNICIPALITY` and `P` (Registered Party)

In [ ]:
grouped = raw_data.groupby(['MUNICIPALITY','P'])
grouped_rejected_ens = raw_data[raw_data['REJRSN'] == "ENS"].groupby(['MUNICIPALITY','P'])

COUNT_NA_COLUMNS = ['ISSDATE', 'RECDATE']
COUNT_VALUE_COLUMNS = ['RECTYPE']

application_counts = grouped['VOTER ID'].count()
rejected_ens_counts = grouped_rejected_ens['REJRSN'].count()
rectype_counts = grouped_rejected_ens['RECTYPE'].value_counts().unstack(fill_value=0)

# Reindex based on counts inded (includes VOTER ID count) to avoid casting to float
all_frames = [application_counts, rejected_ens_counts, rectype_counts]
reindexed_frames = [frame.reindex(application_counts.index, fill_value=0) for frame in all_frames]

# Combine into one table
combined = pd.concat(reindexed_frames, axis=1)
combined.head()


## Rename Columns to be cleaner

In [ ]:
rename_columns = {
    "VOTER ID": "APPS",
    "REJRSN": "REJRSN/ENS",
}
renamed = combined.rename(columns=rename_columns)
renamed.head()

## Add Statewide Section

In [ ]:
state_totals = renamed.unstack(fill_value=0).sum(0).to_frame().unstack(0)
state_totals.index = pd.MultiIndex.from_tuples([("AA_STATEWIDE", party) for party in state_totals.index])
state_totals.index.names = ["MUNICIPALITY", "P"]
state_totals.columns = state_totals.columns.droplevel(0)

state_totaled = pd.concat([state_totals, renamed])
state_totaled.head(10)

## Add Total Rows for each Municipality

In [ ]:
town_sums = state_totaled.groupby("MUNICIPALITY").sum()
town_sums["P"] = "_TOT"
town_sums = town_sums.set_index("P", append=True)
with_sums = pd.concat([state_totaled, town_sums]).sort_index(level=0)
with_sums.head(10)

## Sanity Checks

In [ ]:
# Check REJECTED (ANC) = sum(<Reject Reasons>)
display(all(with_sums['REJRSN/ENS'] == with_sums[['DV', 'IF', 'MA', 'TP', 'VP']].sum(1)))

## Remove Towns with < 5 ANC Rejections

In [ ]:
municipalities = with_sums.index.levels[0]
municipalities_with_rejections = with_sums[with_sums['REJRSN/ENS'] >= 0].\
    index.get_level_values('MUNICIPALITY').unique().tolist()

with_rejections = with_sums.loc[municipalities_with_rejections]
f'Kept {len(municipalities_with_rejections)} of {len(municipalities)} Municipalities'

## Add percents to columns

In [ ]:
APPS_PERCENT_COLUMNS = ['REJRSN/ENS']
REJ_PERCENT_COLUMNS = ['DV', 'IF', 'MA', 'TP', 'VP']
PERCENT_COLUMNS = APPS_PERCENT_COLUMNS + REJ_PERCENT_COLUMNS

apps_fractions = with_rejections[APPS_PERCENT_COLUMNS].div(with_rejections['APPS'], axis=0)
rej_fractions = with_rejections[REJ_PERCENT_COLUMNS].div(with_rejections['REJRSN/ENS'], axis=0)
fractions = pd.concat([apps_fractions, rej_fractions], axis=1).fillna(0)
with_fractions = pd.concat([with_rejections, fractions], axis=1, keys=["NUM", "%"])

# rearrange index to be user-friendly
swapped = with_fractions.swaplevel(0,1, axis=1)
new_index = pd.MultiIndex.from_tuples([('APPS', 'NUM'), *[(col, attr) for col in PERCENT_COLUMNS for attr in ['NUM', '%']]])
reindexed = swapped.reindex(columns=new_index)


## Format and save to file

In [ ]:
styled = reindexed.style.format(
    dict(zip([(col, '%') for col in PERCENT_COLUMNS], ['{:,.2%}'.format] * len(PERCENT_COLUMNS)))
)

if OUTPUT_FORMAT == 'inline':
    display(styled)
elif OUTPUT_FORMAT == 'xlsx':
    path = pathlib.Path(OUTPUT_FILENAME)
    path.parents[0].mkdir(parents=True, exist_ok=True)
    styled.to_excel(str(path))